In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import random

ModuleNotFoundError: No module named 'pandas'

# Projet: Modèles linéaires:  Adaline et Regression Logistique

Nous allons nous intéresser à l'implémentation d'un algorithme de descente de gradient pour trouver le meilleur paramètre d'un module Adaline ou de regression logistique.

Pout cela, on implémentera un algorithme de descente de gradient stochastique que nous avons vu au TP précédent et dont le pseudo-code peut être résumé comme suit:

```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    choisir un exemple (x,y) de Train de façon aléatoire
    calculer h = w*x
    calculer Loss(h, y)
    w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
output: w
```
où "eta" est le pas de la descente de gradient (exemple: eta=0.01).

Si on veut imprimer l'erreur tous les "m" pas de gradient:
```input: Train, eta, m, MaxEp, modele
init : w
epoque=0
while epoque<=MaxEp
    err = 0
    for i in range(m):
        choisir un exemple (x,y) de Train de façon aléatoire
        calculer h = w*x
        err += Loss(h, y)
        w <- w - eta*"gradient de Loss(h, y) par rapport à w"
    epoque <- epoque+1
    print(err)
output: w
```

Pour un poids $w$, on définit $h_\mathbf{w}(\mathbf{x})=w_0x_0+w_1x_1+...w_dx_d$. Pour chacun des deux modèles, et pour un exemple $(\mathbf{x},y)$, la prédiction $\hat{y}(\mathbf{w}, \mathbf{x})$ et la fonction de coût  $\mathcal{L}(\mathbf{w}, \mathbf{x})$ sont: 
- Adaline: $\hat{y}(\mathbf{w}, \mathbf{x}) = h_\mathbf{w}(x)$ et $$\mathcal{L}(\mathbf{w})=(y-\hat{y}(\mathbf{w},\mathbf{x}))^2=(y-h_\mathbf{w}(\mathbf{x}))^2,$$
- Régression logistique: $\hat{y}(w, x) = 1/(1+e^{-h_{\mathbf{w}}(\mathbf{x})})$ et $$\mathcal{L}(\mathbf{w}, x) = - y \log \hat{y}(\mathbf{w},\mathbf{x}) - (1-y)\log(1-\hat{y}(\mathbf{w},\mathbf{x})) = \log(1+e^{h_{\mathbf{w}}(\mathbf{x})})-yh_\mathbf{w}(\mathbf{x}),$$

Nous avons vu les gradients de ces fonctions en TD.

## Partie 1: implémentation de l'algorithme et exemple du "ET logique"

<font color='red'><b>Question 1:</b> le "ET logique".</font> Créer une liste de 4 éléments où chaque élément est un couple de la forme `[x,y]`, avec `x=[1,x1,x2]` et `y = x1 and x2`. Il y a 4 éléments car `x1` et `x2` peuvent chacun prendre la valeur `0` ou `1` (chacun de ces 4 éléments est une liste dont le premier élément est les attributs de l'exemple et le deuxième élément est la classe de l'exemple).

In [56]:
liste = [0] * 4
for x1 in range(2):
    for x2 in range(2):
        liste[x1*1+x2*2] = [[1,x1,x2], x1 and x2] 
liste

[[[1, 0, 0], 0], [[1, 1, 0], 0], [[1, 0, 1], 0], [[1, 1, 1], 1]]

<font color='red'><b>Question 2:</b></font> Coder le modèle Adaline et le modèle de régression logistique et le faire tourner sur le modèle de "ET logique". Calculer le taux d'erreur de votre algorithme sur cette base (où une erreur est comptabilisé si la prédiction est plus proche de la fausse classe que de la vraie classe). 


In [23]:
def prediction(w,x): #hw(x)
    return w@x

def adaline(p,y):
    return (y-p)**2

def regression_logistique(p,y):
    return -y*h + np.log(1+np.exp(-p))
    
def gradient(l, w, x, y):
    if l == adaline:
        return 2*(prediction(w, x)-y)*x
    elif l == regression_logistique:
        return (prediction(w,x)-y)*x
    else:
        exit(1)
    
def eta_001(t):
    return 0.01
    
def SGD(X, Y, Maxep, eta, loss_function):
    w = np.random.rand(3)*0.01
    for epoque in range(Maxep):
        i = rd.randint(0, n - 1)
        p = prediction(w,X[i])
        print(loss_function(p,Y[i]))
        w = w - eta(t) * gradient()
    return w

## Partie 2: premiers tests avec une base de donnée réelle

<font color='red'><b>Question 3:</b></font> Nous allons maintenant nous intéresser au comportement de ces modèles sur la base SONAR de la collection UCI (http://archive.ics.uci.edu/ml/index.php). Cette base contient 208 exemples en dimension 60 séparés par `,` et la dernière élément correspond à la classe de l'exemple.

    1. Télécharger la collection avec la fonction read_table de la librairie pandas (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html). Les options nécessaires sont `sep=','` et `header=None`  
    2. Créer une liste de listes correspondant à la collection; pour cela initialiser la première liste et en parcourant chaque ligne de la matrice de données; créer une liste associée en remplaçant le dernier élément par `-1` ou `+1` et insérer la dans la première liste. 
    Indication: Utiliser la fonction `loc`. 
    3. Écrire une fonction qui génère deux listes de données `x_train` (75%) and `x_test` (25%) en la mélangeant aléatoirement au préalable (indication: on pourra utiliser les fonctions `shuffle` de la librairie `random` et `train_test_split` de la librairie `sklearn.model_selection`)
    

In [60]:
df = pd.read_table('Data/sonar.all-data', sep = ',', header = None)
len(df)
X = np.zeros((len(df), 60))

for i in range(60):
    X[:,i] = df[i]
    
# df
# Y = np.zeros((1))
# for i in range(len(df)):
#     if df[i] == R: Y[i] = 1
#     else: Y[i] = -1

df[60]


0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

<font color='red'><b>Question 4:</b></font> Appliquer ces modèles sur cette base en prenant comme $MaxEp=500$, le pas d'apprentissage $\eta=0.1$ et en choisissant les bases Train et Test de façon aléatoire; Reporter l'erreur moyenne de ces modèles obtenues sur les 20 bases Test?  Refaire l'opération 3 fois avec trois randomisations différentes. 


  | Collection | Adaline     | Régression Logistique |
  |------------|-------------|-----------------------|
  |   SONAR (réplica 1)   |             |                       |
  |   SONAR (réplica 2)   |             |                       |
  |   SONAR (réplica 3)   |             |                       |


## Partie 3: normalisation

Nous allons étudier l'impact de la nomralisation sur les prédictions. Pour cela nous considérons deux stratégies de normalisation communément utilisées dans la littérature:
* Stratégie <i>max</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la valeur maximale de cette caractéristiques
* Stratégie <i>norme</i>: consiste à normaliser chaque caractéristique du vecteur réprésentatif d'une observation par la norme de ce vecteur.

Nous considérons ces trois autres collections de la base UCI:

        * https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29
        * https://archive.ics.uci.edu/ml/datasets/spambase
        * https://archive.ics.uci.edu/ml/datasets/ionosphere

<font color='red'><b>Question 5:</b></font> Ecrire une fonction qui prend en entrée la collection des données et qui retourne la collections normalisée suivant les stratégies <i>max</i> et <i>norme</i>. 
        

<font color='red'><b>Question 6:</b></font> Compléter les tableaux comparatifs suivants en repertant les erreurs moyennes sur 20 lancements des modèles de l'Adaline et de la Régression Logistique et pour les trois cas:

 '*' Les vecteurs ne sont pas normalisés
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

 
 $^n$ Normalisation suivant la stratégie <i>norme</i>
     
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |

  
 $^m$ Normalisation suivant la stratégie <i>max</i>
    
  | Collection |   Adaline   |  Régression Logistique |
  |------------|-------------|------------------------|
  |   BREAST   |             |                        |
  |   IONO     |             |                        |
  |   SONAR    |             |                        |
  |   SPAM     |             |                        |
